# Capstone Week 3
---

# Index
- [Capstone Objectives](#Capstone-Objectives)
- [Read in Data](#Read-in-Data)
    - [Merge 2018 and 2019](#Merge-2018-and-2019)
    - [Make advisor dictionary mapper](#Make-advisor-dictionary-mapper)
- [Data Cleaning](#Data-Cleaning)
    - [Custom Cleaning Functions](#Custom-Cleaning-Functions)
    - [Create Cleaning Pipeline](#Create-Cleaning-Pipeline)
- [Model building](#Model-building)
- [Make predictions](#Make-predictions)
- [Residuals](#Residuals)
- [Model Interpretation](#Model-Interpretation)

# Capstone Objectives
- Assist sales and marketing by improving their targeting
- Predict sales for 2019 using the data for 2018

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 50)

[Back to Top](#Index)
# Read in Data

In [ ]:
df18 = pd.read_excel("../Transaction Data.xlsx", sheet_name="Transactions18")
df19 = pd.read_excel("../Transaction Data.xlsx", sheet_name="Transactions19")

## Merge 2018 and 2019

In [ ]:
df = pd.merge(
    df18, 
    df19, 
    on='CONTACT_ID',
    suffixes=['_2018', '_2019']
)
print(df.shape)

## Make advisor dictionary mapper

In [ ]:
adviser_lookup = {idx: contact_id for idx, contact_id in enumerate(df['CONTACT_ID'])}

[Back to Top](#Index)
# Data Cleaning

In [ ]:
# make a variable to keep all of the columns we want to drop
COLS_TO_DROP = [
    'refresh_date_2019', 'refresh_date_2018', 'CONTACT_ID',
]

COLS_TO_KEEP = [
    'no_of_sales_12M_1', 'no_of_Redemption_12M_1', 'no_of_sales_12M_10K',
    'no_of_Redemption_12M_10K', 'no_of_funds_sold_12M_1',
    'no_of_funds_redeemed_12M_1', 'no_of_fund_sales_12M_10K',
    'no_of_funds_Redemption_12M_10K', 'no_of_assetclass_sold_12M_1',
    'no_of_assetclass_redeemed_12M_1', 'no_of_assetclass_sales_12M_10K',
    'no_of_assetclass_Redemption_12M_10K', 'No_of_fund_curr',
    'No_of_asset_curr', 'AUM', 'sales_curr', 'sales_12M_2018',
    'redemption_curr', 'redemption_12M', 'new_Fund_added_12M_2018',
    'aum_AC_EQUITY', 'aum_AC_FIXED_INCOME_MUNI',
    'aum_AC_FIXED_INCOME_TAXABLE', 'aum_AC_MONEY', 'aum_AC_MULTIPLE',
    'aum_AC_PHYSICAL_COMMODITY', 'aum_AC_REAL_ESTATE', 'aum_AC_TARGET',
    'aum_P_529', 'aum_P_ALT', 'aum_P_CEF', 'aum_P_ETF', 'aum_P_MF',
    'aum_P_SMA', 'aum_P_UCITS', 'aum_P_UIT'
]

In [ ]:
X = df.drop(['sales_12M_2019', 'new_Fund_added_12M_2019'], axis=1)
y = df['sales_12M_2019']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=24)

## Custom Cleaning Functions

In [ ]:
def extract_columns(df):
    '''extract out columns not listed in COLS_TO_DROP variable'''
    cols_to_keep = [col for col in df.columns if col not in COLS_TO_DROP]
    return df.loc[:, cols_to_keep].copy()


def fillna_values(df):
    '''fill nan values with zero'''
    if isinstance(df, type(pd.Series(dtype='float64'))):
        return df.fillna(0)
    num_df = df.select_dtypes(include=['number']).fillna(0)
    non_num_df = df.select_dtypes(exclude=['number'])
    return pd.concat([num_df, non_num_df], axis=1)

[Back to Top](#Index)
## Create Cleaning Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
extract_columns_trans = FunctionTransformer(extract_columns, validate=False)
fillna_values_trans = FunctionTransformer(fillna_values, validate=False)

Make pipeline for target variable

In [ ]:
targ_pipe = Pipeline([
    ('fillna_values_trans', fillna_values_trans),
])

In [ ]:
y_train = targ_pipe.fit_transform(y_train) # fit and transform TRAINING

In [ ]:
y_test = targ_pipe.transform(y_test) # transform only TESTING

Make pipeline for features

In [ ]:
feat_pipe = Pipeline([
    ('extract_columns_trans', extract_columns_trans),
    ('fillna_values_trans', fillna_values_trans),
    ('StandardScaler', StandardScaler()),
])

In [ ]:
train_array = feat_pipe.fit(X_train, y_train).transform(X_train)
train_array[:2]

Convert the preprocessed numpy array back to pandas dataframe

In [ ]:
X_train_prepared = pd.DataFrame(
    feat_pipe.fit(X_train, y_train).transform(X_train),
    index=X_train.index,
    columns=COLS_TO_KEEP
)

In [ ]:
X_train_prepared.head()

**TRANSFORM** the test set (Do NOT fit the pipeline on testing!)

In [ ]:
X_test_prepared = pd.DataFrame(
    pipe.fit(X_test, y_test).transform(X_test),
    index=X_test.index,
    columns=COLS_TO_KEEP
)

In [ ]:
X_test_prepared.head()

[Back to Top](#Index)
# Model building

Look at 1hr mark in Jacob's 3rd OH session
- Grid search CV with multiple models in there (Linear Reg, ElasticNet, RF, XGBoost)
- SelectFromModel

Look at AML Capstone to see if you have ideas for improving data quality.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb

In [ ]:
feat_pipe = Pipeline([
    ('extract_columns_trans', extract_columns_trans),
    ('fillna_values_trans', fillna_values_trans),
    ('StandardScaler', StandardScaler()),
    ('select', SelectFromModel(Ridge(), max_features=10)),
    ('forest', RandomForestRegressor()),
])

In [ ]:
feat_pipe.fit(X_train, y_train)

[Back to Top](#Index)
# Make predictions

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cross_validate(feat_pipe, X_train, y_train)

In [ ]:
#  make regression predictions
y_train_preds = feat_pipe.predict(X_train_prepared)
y_test_preds = feat_pipe.predict(X_test_prepared)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
ax.scatter(x=y_train, y=y_train_preds, alpha=0.4)
ax.plot([0, 20000000], [0,20000000])
ax.set_xlim([0, 20000000])
ax.set_ylim([0, 20000000])
ax.set_title("Actual vs Predicted - Regression")
ax.set_xlabel("Actual")
ax.set_ylabel("Predicted");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8))
ax.scatter(x=y_test, y=y_test_preds, alpha=0.4)
ax.plot([0, 20000000], [0,20000000])
ax.set_xlim([0, 20000000])
ax.set_ylim([0, 20000000])
ax.set_title("Actual vs Predicted - Regression")
ax.set_xlabel("Actual")
ax.set_ylabel("Predicted");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))
y_train.hist(bins=30, ax=ax);

[Back to Top](#Index)
# Residuals

In [ ]:
# get residuals
residuals = y_test_preds - y_test

In [ ]:
# plot predictions vs residuals
fig, axes = plt.subplots(2,2,figsize=(14,10))

# plot scatter on upper right plot
axes[0,0].scatter(x=y_test_preds, y=residuals, alpha=0.5)
axes[0,0].set(xlabel="Residuals",ylabel="Predictions");

# plot hist on upper left plot
axes[0,1].hist(residuals, bins=50)
axes[0,1].set(xlabel='Residuals', ylabel='Frequency');

In [ ]:
from statsmodels.api import qqplot

In [ ]:
axes[1,0].set_ylim([-3.5, 3.5])
axes[1,0].set_xlim([-3.5, 3.5])

In [ ]:
qqplot(residuals, fit=True, line='r', ax=axes[1,0])

[Back to Top](#Index)
# Model Interpretation

# Scratch Work

In [ ]:
param_grid = {
    'select__estimator': [Ridge(), RandomForestRegressor(), xgb.XGBRegressor()],
    'select__max_features': [2, 5, 7, 9, 11, 13],
    'forest__n_estimators': [25, 50, 75, 100, 150, 200],
    'forest__max_features': ['sqrt', 'log2'],
}

In [ ]:
rs = RandomizedSearchCV(
    feat_pipe, 
    param_distributions=param_grid,
    cv=3,
    verbose=2
)

In [ ]:
rs.fit(feat_pipe, X_train, y_train)